In [5]:
import pennylane as qml
from pennylane import numpy as np

import matplotlib.pyplot as plt


# == Hamiltonain setup == 
Coeff_ham = [1]
Obs = [qml.PauliZ(0)]
Hamiltonian = qml.Hamiltonian(Coeff_ham, Obs)

dev = qml.device("default.mixed", wires = 1)

t = 0.0001
gamma_dephase = 1 - np.exp(-2 * t)


gamma_ps = 0.8

# B, t = 1 * 10**-6, 0.5 * 10**
# phi = gamma_ps * np.exp(1) * 

# Stage_1: Generate dephase
@qml.qnode(dev)
def Dephase_circuit(phi):
    qml.Hadamard(wires=0)
    
    # Time_evo t fixed as -phi/2
    qml.ApproxTimeEvolution(Hamiltonian, -phi/2, 1)
    
    qml.PhaseDamping(gamma_dephase, wires = 0)
    
    qml.Hadamard(wires=0)
    
    return qml.density_matrix(wires=0)

# Stage_2: Post-selection
@qml.qnode(dev)
def Post_selection(phi):
    rho_dephase = Dephase_circuit(phi)
    Kraus_oper = np.array([ [np.sqrt(1-gamma_ps), 0], [0, 1] ])
    
    rho_ps = (Kraus_oper @ rho_dephase @ Kraus_oper.conj().T) / (np.trace(Kraus_oper @ rho_dephase @ Kraus_oper.conj().T))
    qml.QubitDensityMatrix(rho_ps, wires=0)
    
    return qml.density_matrix(wires=0)